In [22]:
%load_ext autoreload
%autoreload 2

from lfp_tools import startup_local
import utils.behavioral_utils as behavioral_utils
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
FEATURE_DIMS = ["Color", "Shape", "Pattern"]


In [39]:
sess_name = "20180802"
behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
beh = pd.read_csv(behavior_path)

valid_beh = behavioral_utils.get_valid_trials(beh)

In [40]:
res = startup_local.get_sac_dataframe('nhp', "SA", "WCST", sess_name, False)

In [41]:
res = res[res.trial.isin(valid_beh.TrialNumber)]

In [42]:
res.obj_end.unique()

array(['-', 'h1', 'h3', 'n', 'h0', 'h2', 's1', 's2', 's0', 's3', 's0_s3',
       's2_s3', 's1_s2', 'h1_s3', 'h2_s3', 'h0_s3', 'h0_s1'], dtype=object)

In [43]:
# saccade was onto something different
other_cards = ["h1", "h2", "h3", "s1", "s2", "s3"]
res = res[res.obj_end.isin(other_cards)]
# saccade was onto something different
res = res[res.obj_start != res.obj_end]

In [44]:
merged = pd.merge(res, valid_beh, left_on="trial", right_on="TrialNumber")

In [45]:
merged

,trial,interval,obj_start,obj_end,distance,direction,stereotypical,time_start,time_end,time_peak,...,Item0Pattern,Item1Shape,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern
0,35,obj,n,h1,4.474433,166.877472,0,929898,929916,929907,...,RIPPLE,TRIANGLE,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER
1,35,obj,h1,h3,11.816526,83.602343,0,930002,930023,930011,...,RIPPLE,TRIANGLE,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER
2,36,obj,h0,h3,11.825025,71.122676,0,937330,937352,937339,...,RIPPLE,TRIANGLE,YELLOW,ESCHER,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT
3,37,obj,n,h3,5.748549,-92.689654,0,944530,944548,944539,...,ESCHER,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL
4,37,obj_fix,h3,h1,1.038441,-166.108002,0,945347,945356,945351,...,ESCHER,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1738,obj_fix,h0,s3,3.264027,124.341964,0,13617041,13617058,13617050,...,RIPPLE,SQUARE,CYAN,ESCHER,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL
3118,1739,obj,h3,h1,15.509239,93.445712,0,13619877,13619903,13619886,...,SWIRL,SQUARE,CYAN,ESCHER,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT
3119,1740,obj,n,h2,6.122174,175.485924,0,13623781,13623802,13623790,...,SWIRL,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE
3120,1740,obj,h2,h1,12.863098,80.463145,0,13623893,13623918,13623906,...,SWIRL,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE


In [46]:
def get_features_from_card(row): 
    card_idx = int(row["obj_end"][1])
    for feature_dim in FEATURE_DIMS:
        row[feature_dim] = row[f"Item{card_idx}{feature_dim}"]
    return row
merged_with_features = merged.apply(get_features_from_card, axis=1)

In [47]:
merged

,trial,interval,obj_start,obj_end,distance,direction,stereotypical,time_start,time_end,time_peak,...,Item0Pattern,Item1Shape,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern
0,35,obj,n,h1,4.474433,166.877472,0,929898,929916,929907,...,RIPPLE,TRIANGLE,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER
1,35,obj,h1,h3,11.816526,83.602343,0,930002,930023,930011,...,RIPPLE,TRIANGLE,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER
2,36,obj,h0,h3,11.825025,71.122676,0,937330,937352,937339,...,RIPPLE,TRIANGLE,YELLOW,ESCHER,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT
3,37,obj,n,h3,5.748549,-92.689654,0,944530,944548,944539,...,ESCHER,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL
4,37,obj_fix,h3,h1,1.038441,-166.108002,0,945347,945356,945351,...,ESCHER,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1738,obj_fix,h0,s3,3.264027,124.341964,0,13617041,13617058,13617050,...,RIPPLE,SQUARE,CYAN,ESCHER,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL
3118,1739,obj,h3,h1,15.509239,93.445712,0,13619877,13619903,13619886,...,SWIRL,SQUARE,CYAN,ESCHER,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT
3119,1740,obj,n,h2,6.122174,175.485924,0,13623781,13623802,13623790,...,SWIRL,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE
3120,1740,obj,h2,h1,12.863098,80.463145,0,13623893,13623918,13623906,...,SWIRL,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE


In [20]:
merged_with_features["IntervalID"] = np.arange(len(merged_with_features))

In [21]:
merged_with_features

,trial,interval,obj_start,obj_end,distance,direction,stereotypical,time_start,time_end,time_peak,...,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern,Color,Shape,Pattern,IntervalID
0,35,obj,n,h1,4.474433,166.877472,0,929898,929916,929907,...,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER,MAGENTA,TRIANGLE,POLKADOT,0
1,35,obj,h1,h3,11.816526,83.602343,0,930002,930023,930011,...,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER,YELLOW,SQUARE,ESCHER,1
2,36,obj,h0,h3,11.825025,71.122676,0,937330,937352,937339,...,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT,MAGENTA,SQUARE,POLKADOT,2
3,37,obj,n,h3,5.748549,-92.689654,0,944530,944548,944539,...,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL,CYAN,SQUARE,SWIRL,3
4,37,obj_fix,h3,h1,1.038441,-166.108002,0,945347,945356,945351,...,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL,YELLOW,TRIANGLE,RIPPLE,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,1738,obj,s2,h2,2.895244,-98.388588,0,13615957,13615974,13615965,...,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL,MAGENTA,STAR,POLKADOT,2731
2732,1739,obj,h3,h1,15.509239,93.445712,0,13619877,13619903,13619886,...,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT,CYAN,SQUARE,ESCHER,2732
2733,1740,obj,n,h2,6.122174,175.485924,0,13623781,13623802,13623790,...,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE,MAGENTA,SQUARE,ESCHER,2733
2734,1740,obj,h2,h1,12.863098,80.463145,0,13623893,13623918,13623906,...,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE,CYAN,TRIANGLE,POLKADOT,2734
